In [280]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [281]:
df = pd.read_csv('../../data/df_rfm.csv')

df

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
0,489434,85048,15CM_CHRISTMAS_GLASS_BALL_20_LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United_Kingdom,83.40
1,489434,22041,"RECORD_FRAME_7""_SINGLE_SIZE",48,2009-12-01 07:45:00,2.10,13085,United_Kingdom,100.80
2,489434,21232,STRAWBERRY_CERAMIC_TRINKET_BOX,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
3,489434,22064,PINK_DOUGHNUT_TRINKET_POT,24,2009-12-01 07:45:00,1.65,13085,United_Kingdom,39.60
4,489434,21871,SAVE_THE_PLANET_MUG,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.00
...,...,...,...,...,...,...,...,...,...
713597,581587,22613,PACK_OF_20_SPACEBOY_NAPKINS,12,2011-12-09 12:50:00,0.85,12680,France,10.20
713598,581587,22899,CHILDREN'S_APRON_DOLLY_GIRL,6,2011-12-09 12:50:00,2.10,12680,France,12.60
713599,581587,23254,CHILDRENS_CUTLERY_DOLLY_GIRL,4,2011-12-09 12:50:00,4.15,12680,France,16.60
713600,581587,23255,CHILDRENS_CUTLERY_CIRCUS_PARADE,4,2011-12-09 12:50:00,4.15,12680,France,16.60


In [282]:
# prend seulement les dates de 2009 et decembre 2010 separer le data frame en deu
df_2009_2010 = df[(df['InvoiceDate'] >= '2009-12-01') & (df['InvoiceDate'] < '2010-12-01')]
df_2011 = df[df['InvoiceDate'] >= '2010-12-01']

In [283]:
df_2009_2010

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer_ID,Country,TotalSales
0,489434,85048,15CM_CHRISTMAS_GLASS_BALL_20_LIGHTS,12,2009-12-01 07:45:00,6.95,13085,United_Kingdom,83.4
1,489434,22041,"RECORD_FRAME_7""_SINGLE_SIZE",48,2009-12-01 07:45:00,2.10,13085,United_Kingdom,100.8
2,489434,21232,STRAWBERRY_CERAMIC_TRINKET_BOX,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.0
3,489434,22064,PINK_DOUGHNUT_TRINKET_POT,24,2009-12-01 07:45:00,1.65,13085,United_Kingdom,39.6
4,489434,21871,SAVE_THE_PLANET_MUG,24,2009-12-01 07:45:00,1.25,13085,United_Kingdom,30.0
...,...,...,...,...,...,...,...,...,...
337794,536364,21175,GIN_+_TONIC_DIET_METAL_SIGN,12,2010-11-30 19:35:00,2.10,14441,United_Kingdom,25.2
337795,536364,21524,DOORMAT_SPOTTY_HOME_SWEET_HOME,2,2010-11-30 19:35:00,7.95,14441,United_Kingdom,15.9
337796,536364,21098,CHRISTMAS_TOILET_ROLL,12,2010-11-30 19:35:00,1.25,14441,United_Kingdom,15.0
337797,536364,21181,PLEASE_ONE_PERSON_METAL_SIGN,24,2010-11-30 19:35:00,2.10,14441,United_Kingdom,50.4


### Préparation de l'Analyse RFM pour 2009-2010:

In [284]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

# Ensure that InvoiceDate is in datetime format
df_2009_2010['InvoiceDate'] = pd.to_datetime(df_2009_2010['InvoiceDate'])

# Calculate Recency, Frequency and Monetary value for each customer
snapshot_date = df_2009_2010['InvoiceDate'].max() + pd.DateOffset(days=1) 

rfm_2009_2010 = df_2009_2010.groupby(['Customer_ID']).agg({
    'InvoiceDate': lambda x: (snapshot_date - x.max()).days,
    'Invoice': 'count',
    'TotalSales': 'sum'})

# Rename the columns 
rfm_2009_2010.rename(columns={'InvoiceDate': 'Recency',
                   'Invoice': 'Frequency',
                   'TotalSales': 'MonetaryValue'}, inplace=True)


/tmp/ipykernel_21393/1952453250.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [285]:
# Apply K-Means Clustering to segment customers
kmeans = KMeans(n_clusters=3, random_state=1) 
kmeans.fit(rfm_2009_2010)
rfm_2009_2010['Cluster'] = kmeans.labels_

/home/apprenant/miniconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [286]:
# Ensure that InvoiceDate is in datetime format
df_2011['InvoiceDate'] = pd.to_datetime(df_2011['InvoiceDate'])

# Calculate Recency, Frequency and Monetary value for each customer
snapshot_date_2011 = df_2011['InvoiceDate'].max() + pd.DateOffset(days=1) 

rfm_2011 = df_2011.groupby(['Customer_ID']).agg({
    'InvoiceDate': lambda x: (snapshot_date_2011 - x.max()).days,
    'Invoice': 'count',
    'TotalSales': 'sum'})

# Rename the columns 
rfm_2011.rename(columns={'InvoiceDate': 'Recency',
                   'Invoice': 'Frequency',
                   'TotalSales': 'MonetaryValue'}, inplace=True)

/tmp/ipykernel_21393/2209789562.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [287]:
# Apply K-Means Clustering to segment customers
kmeans_2011 = KMeans(n_clusters=4, random_state=1) 
kmeans_2011.fit(rfm_2011)
rfm_2011['Cluster'] = kmeans_2011.labels_

/home/apprenant/miniconda3/envs/machine-learning/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



In [288]:
# Extract the cluster labels for 2010 and 2011 customers
labels_2010 = rfm_2009_2010['Cluster']
labels_2011 = rfm_2011['Cluster']

# Calculate the Adjusted Rand Index
ari = adjusted_rand_score(labels_2010, labels_2011)
print(f'Adjusted Rand Index: {ari}')

ValueError: Found input variables with inconsistent numbers of samples: [4198, 4311]